One challenge with the DIVVY data is that it comes in several different forms.
I've downloaded and unzipped all the data from 2017 to 2021.
The following shows that there are 9 different 

In [136]:
import glob
workDir = '/home/alp/Google Drive/Python/DataSets/Data_Divvy/CVS_archive'

file_names = sorted(glob.glob(workDir+'Divvy*.csv'))

cols = []
line = ''

for file in file_names:
    with open(file) as f:
        newln = f.readline()
        if newln != line:
            cols.append((file.replace(workDir,''),newln))
            line = newln
        

blech - this is still hard to read. Let's sort the list by the variable lists and then find the list of unique variable names, so we can compare them more easily.

In [23]:
c_cols = []
cStr = ''
cols.sort(key = lambda x: x[1])
for c in cols:
    if c[1] != cStr:
        c_cols.append(c)
        cStr = c[1]

In [24]:
c_cols.sort(key = lambda x: x[0])
print(len(c_cols),c_cols)

4 [('202004-divvy-tripdata.csv', 'ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual\n'), ('Divvy_Trips_2017_Q1.csv', '"trip_id","start_time","end_time","bikeid","tripduration","from_station_id","from_station_name","to_station_id","to_station_name","usertype","gender","birthyear"\n'), ('Divvy_Trips_2017_Q4.csv', 'trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear\n'), ('Divvy_Trips_2018_Q1.csv', '01 - Rental Details Rental ID,01 - Rental Details Local Start Time,01 - Rental Details Local End Time,01 - Rental Details Bike ID,01 - Rental Details Duration In Seconds Uncapped,03 - Rental Start Station ID,03 - Rental Start Station Name,02 - Rental End Station ID,02 - Rental End Station Name,User Type,Member Gender,05 - Member Details Member Birthday Year\n')]


OK, so while the columns are set up with different names, the data from 2017-2020Q1 all uses the same number and arrangmeent of columns:
1. trip_id
1. start_time
1. end_time
1. bikeid
1. tripduration
1. from_station_id
1. from_station_name
1. to_station_id
1. to_station_name
1. usertype
1. gender
1. birthyear

Before 2020_04, the data is saved in quartly chunks, using a "YYYYMM-divvy-tripdata.csv" columns are as follows
1. ride_id
1. rideable_type
1. started_at
1. ended_at
1. start_station_name
1. start_station_id
1. end_station_name
1. end_station_id
1. start_lat
1. start_lng
1. end_lat
1. end_lng
1. member_casual

For my purposes, the most important data that is missing from pre-2020 data is lat/long data for the stations. It's also annoying that it uses a file naming convention. Let's process all the quarterly data to add the lat/long data and save it as monthly JSON files.

In [11]:
import pandas as pd
from sodapy import Socrata

#https://data.cityofchicago.org/Transportation/Divvy-Bicycle-Stations/bbyy-e7gq/data

 
print('Enter App Token')
appToken = input()

client=Socrata('data.cityofchicago.org',appToken)
results=client.get('bbyy-e7gq',limit=5000,content_type= 'json')
stations = pd.DataFrame.from_dict(results)

#url = 'https://data.cityofchicago.org/resource/bbyy-e7gq.json?$$app_token=appToken$limit=10000'
#stations = pd.read_json(url)


Enter App Token


In [12]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369 entries, 0 to 1368
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           1369 non-null   object
 1   station_name                 1369 non-null   object
 2   total_docks                  1369 non-null   object
 3   docks_in_service             1369 non-null   object
 4   status                       1369 non-null   object
 5   latitude                     1369 non-null   object
 6   longitude                    1369 non-null   object
 7   location                     1369 non-null   object
 8   :@computed_region_awaf_s7ux  1355 non-null   object
 9   :@computed_region_6mkv_f3dw  1369 non-null   object
 10  :@computed_region_vrxf_vc4k  1355 non-null   object
 11  :@computed_region_bdys_3d7i  1355 non-null   object
 12  :@computed_region_43wa_7qmu  1355 non-null   object
dtypes: object(13)
memory usage: 139.2

In [43]:
stations.head()

,id,station_name,total_docks,docks_in_service,status,latitude,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu
0,2,Buckingham Fountain (Temp),39,39,In Service,41.869620747,-87.623981237,"{'latitude': '41.86962074748123', 'longitude':...",48,14913,38,368,10
1,3,Shedd Aquarium,55,55,In Service,41.867225957,-87.61535539,"{'latitude': '41.86722595682', 'longitude': '-...",48,14913,34,374,10
2,4,Burnham Harbor,23,23,In Service,41.857411787,-87.613791525,"{'latitude': '41.85741178707404', 'longitude':...",48,21194,34,374,10
3,5,State St & Harrison St,23,23,In Service,41.874053,-87.627716,"{'latitude': '41.874053', 'longitude': '-87.62...",48,14913,38,12,10
4,6,Dusable Harbor,39,39,In Service,41.886976,-87.612813,"{'latitude': '41.886976', 'longitude': '-87.61...",22,21182,38,159,36


In [44]:
stations['id'] = pd.to_numeric(stations['id'])
stations['latitude'] = pd.to_numeric(stations['latitude'])
stations['longitude'] = pd.to_numeric(stations['longitude'])
stations.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369 entries, 0 to 1368
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           1369 non-null   int64  
 1   station_name                 1369 non-null   object 
 2   total_docks                  1369 non-null   object 
 3   docks_in_service             1369 non-null   object 
 4   status                       1369 non-null   object 
 5   latitude                     1369 non-null   float64
 6   longitude                    1369 non-null   float64
 7   location                     1369 non-null   object 
 8   :@computed_region_awaf_s7ux  1355 non-null   object 
 9   :@computed_region_6mkv_f3dw  1369 non-null   object 
 10  :@computed_region_vrxf_vc4k  1355 non-null   object 
 11  :@computed_region_bdys_3d7i  1355 non-null   object 
 12  :@computed_region_43wa_7qmu  1355 non-null   object 
dtypes: float64(2), int

In [172]:
def processFile(file,stations):
# read the CSV file of trip data from the provided file name,
# parsing the two date-time columns as dates,
# renaming all columns to standard names, 
# merge it with the station data. The first merge adds lat/long for 'from station'; the second adds lat/long for the 'to station'
# the merge uses the station ID as the item to match on
# rename the columns to match those of the post-April 2020 files
    trip_data = pd.read_csv(file,parse_dates=[1,2])
    trip_data.columns=['trip_id','start_time','end_time','bikeid','tripduration',
        'from_station_id','from_station_name','to_station_id','to_station_name',
        'usertype','gender','birthyear']
    trip_data = pd.merge(trip_data,stations[['id','latitude','longitude']],'left', left_on = 'from_station_id',right_on = 'id')
    trip_data = pd.merge(trip_data,stations[['id','latitude','longitude']],'left',left_on = 'to_station_id',right_on = 'id')
    trip_data = trip_data.drop(columns = ['id_x','id_y'])
    trip_data = trip_data.rename(columns={'latitude_x':'start_lat','longitude_x':'start_lng', 'latitude_y':'end_lat','longitude_y':'end_lng'})

    # create grouby frames based on the month
    #months = trip_data.groupby(trip_data.start_time.dt.month)
    months = trip_data.groupby(pd.Grouper(key = 'start_time', freq='MS'))

    workDir = '/home/alp/Google Drive/Python/DataSets/Data_Divvy/'
    fn = file.replace(workDir,'')
    
    for key, gp in months:
        fn = key.strftime('%Y') + key.strftime('%m')
        saveFile = workDir + 'monthly/' + fn +'-divvy-tripdata.json'
        gp.to_json(saveFile,orient='index')
        print(saveFile)

#    file_dates = trip_data['start_time'].dt.strftime("%Y%m").unique().tolist()
    

Because there's something glitchy the data, I had to feed the data files into the procedure one-by-one.
2017 Q1-Q3 had problems.
2020 Q1 had a problem.

In [ ]:
processFile(file_names[1],stations)


Let's look a them and get them read correctly.


In [189]:
fileN = file_names[0]


In [191]:
td = pd.read_csv(fileN)

In [195]:
td.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,13518905,3/31/2017 23:59:07,4/1/2017 00:13:24,5292,857,66,Clinton St & Lake St,171,May St & Cullerton St,Subscriber,Male,1989.0
1,13518904,3/31/2017 23:56:25,4/1/2017 00:00:21,4408,236,199,Wabash Ave & Grand Ave,26,McClurg Ct & Illinois St,Subscriber,Male,1990.0
2,13518903,3/31/2017 23:55:33,4/1/2017 00:01:21,696,348,520,Greenview Ave & Jarvis Ave,432,Clark St & Lunt Ave,Subscriber,Female,1979.0
3,13518902,3/31/2017 23:54:46,3/31/2017 23:59:34,4915,288,110,Dearborn St & Erie St,142,McClurg Ct & Erie St,Subscriber,Male,1985.0
4,13518901,3/31/2017 23:53:33,4/1/2017 00:00:28,4247,415,327,Sheffield Ave & Webster Ave,331,Halsted St & Blackhawk St (*),Subscriber,Female,1989.0


OK, the problem is the time columns (1,2) are using slashe, not dashes, do the date parser is hanging.
We'll adjust these manually for each file

In [206]:
fileN = file_names[2]
td = pd.read_csv(fileN)

td['start_time'] = pd.to_datetime(td['start_time'],format = '%m/%d/%Y %H:%M:%S')
td['end_time'] = pd.to_datetime(td['end_time'],format = '%m/%d/%Y %H:%M:%S')

In [207]:
trip_data = td
trip_data.columns=['trip_id','start_time','end_time','bikeid','tripduration',
    'from_station_id','from_station_name','to_station_id','to_station_name',
    'usertype','gender','birthyear']
trip_data = pd.merge(trip_data,stations[['id','latitude','longitude']],'left', left_on = 'from_station_id',right_on = 'id')
trip_data = pd.merge(trip_data,stations[['id','latitude','longitude']],'left',left_on = 'to_station_id',right_on = 'id')
trip_data = trip_data.drop(columns = ['id_x','id_y'])
trip_data = trip_data.rename(columns={'latitude_x':'start_lat','longitude_x':'start_lng', 'latitude_y':'end_lat','longitude_y':'end_lng'})

# create grouby frames based on the month
#months = trip_data.groupby(trip_data.start_time.dt.month)
months = trip_data.groupby(pd.Grouper(key = 'start_time', freq='MS'))

workDir = '/home/alp/Google Drive/Python/DataSets/Data_Divvy/'

for key, gp in months:
    fn = key.strftime('%Y') + key.strftime('%m')
    saveFile = workDir + 'monthly/' + fn +'-divvy-tripdata.json'
    gp.to_json(saveFile,orient='index')
    print(saveFile)



/home/alp/Google Drive/Python/DataSets/Data_Divvy/monthly/201707-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/monthly/201708-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/monthly/201709-divvy-tripdata.json


Lets look at 2020 Q1

In [4]:
import pandas as pd
workDir = '/home/alp/Google Drive/Python/DataSets/Data_Divvy/'
fileN = 'Divvy_Trips_2020_Q1.csv'
td = pd.read_csv(workDir + fileN,parse_dates=[1,2])
td.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426887 entries, 0 to 426886
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             426887 non-null  object        
 1   rideable_type       426887 non-null  object        
 2   started_at          426887 non-null  datetime64[ns]
 3   ended_at            426887 non-null  object        
 4   start_station_name  426887 non-null  object        
 5   start_station_id    426887 non-null  int64         
 6   end_station_name    426886 non-null  object        
 7   end_station_id      426886 non-null  float64       
 8   start_lat           426887 non-null  float64       
 9   start_lng           426887 non-null  float64       
 10  end_lat             426886 non-null  float64       
 11  end_lng             426886 non-null  float64       
 12  member_casual       426887 non-null  object        
dtypes: datetime64[ns](1), float64

So this file is using the 2020 new column names, including the new 'ridable_type' and the 'member_casual'
So we need to read it again with the proper cols identified as dates
Since it's alreayd got lat/long data, we can then just break it into months

In [9]:
fileN = 'Divvy_Trips_2020_Q1.csv'
td = pd.read_csv(workDir + fileN,parse_dates=[2,3])

trip_data = td

months = trip_data.groupby(pd.Grouper(key = 'started_at', freq='MS'))

workDir = '/home/alp/Google Drive/Python/DataSets/Data_Divvy/'

for key, gp in months:
    fn = key.strftime('%Y') + key.strftime('%m')
    saveFile = workDir + fn +'-divvy-tripdata.json'
    gp.to_json(saveFile,orient='index')
    print(saveFile)


/home/alp/Google Drive/Python/DataSets/Data_Divvy/202001-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202002-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202003-divvy-tripdata.json


In [10]:
pd.DatetimeIndex(td['started_at']).month.value_counts().sort_index()

1    143884
2    139585
3    143418
Name: started_at, dtype: int64

In [212]:
td.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426887 entries, 0 to 426886
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             426887 non-null  object        
 1   rideable_type       426887 non-null  object        
 2   started_at          426887 non-null  datetime64[ns]
 3   ended_at            426887 non-null  datetime64[ns]
 4   start_station_name  426887 non-null  object        
 5   start_station_id    426887 non-null  int64         
 6   end_station_name    426886 non-null  object        
 7   end_station_id      426886 non-null  float64       
 8   start_lat           426887 non-null  float64       
 9   start_lng           426887 non-null  float64       
 10  end_lat             426886 non-null  float64       
 11  end_lng             426886 non-null  float64       
 12  member_casual       426887 non-null  object        
dtypes: datetime64[ns](2), float64

In [214]:
fileN = workDir + '202004-divvy-tripdata.csv'
td = pd.read_csv(fileN,parse_dates=[2,3])
td.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84776 entries, 0 to 84775
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ride_id             84776 non-null  object        
 1   rideable_type       84776 non-null  object        
 2   started_at          84776 non-null  datetime64[ns]
 3   ended_at            84776 non-null  datetime64[ns]
 4   start_station_name  84776 non-null  object        
 5   start_station_id    84776 non-null  int64         
 6   end_station_name    84677 non-null  object        
 7   end_station_id      84677 non-null  float64       
 8   start_lat           84776 non-null  float64       
 9   start_lng           84776 non-null  float64       
 10  end_lat             84677 non-null  float64       
 11  end_lng             84677 non-null  float64       
 12  member_casual       84776 non-null  object        
dtypes: datetime64[ns](2), float64(5), int64(1), ob

In [217]:
fileN[:-3]+'json'

'/home/alp/Google Drive/Python/DataSets/Data_Divvy/202004-divvy-tripdata.json'

In [18]:
def process2020(file,stations):
    # this is easier: we need only read the file, convert the date-tmes into DT stamps
    # since it's already saved as a month file, I don't need to mess about with the grouper either
    # just  output it as a json file
    trip_data = pd.read_csv(file,parse_dates=[2,3])

    saveFile = file[:-3]+'json'
    trip_data.to_json(saveFile,orient='index')
    print(saveFile) 

In [20]:
import glob
file_names = sorted(glob.glob(workDir+'2021*.csv'))

for file in file_names:
    process2020(file,stations)

/home/alp/Google Drive/Python/DataSets/Data_Divvy/202101-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202102-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202103-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202104-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202105-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202106-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202107-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202108-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202109-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202110-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202111-divvy-tripdata.json
/home/alp/Google Drive/Python/DataSets/Data_Divvy/202112-divvy-tripdata.json
